In [1]:
%pylab inline
from astropy.io import fits
import astropy.units as u
import pickle as pickle
import os as os
import pandas as pd

from spec_utils import read_irs

from spectools_ir.utils import extract_hitran_data, spec_convol, make_rotation_diagram, get_molmass
from spectools_ir.utils import compute_thermal_velocity, sigma_to_fwhm, fwhm_to_sigma, wn_to_k, spec_convol_R
from spectools_ir.utils import get_miri_mrs_resolution, get_miri_mrs_wavelengths, make_miri_mrs_figure

from spectools_ir.flux_calculator import calc_fluxes, make_lineshape

from spectools_ir.slabspec import make_spec

from spectools_ir.slab_fitter import Config, LineData,Retrieval
from spectools_ir.slab_fitter import corner_plot, trace_plot, find_best_fit, compute_model_fluxes
from spectools_ir.slab_fitter import calc_solid_angle, calc_radius
from spectools_ir.slab_fitter import read_data_from_file, get_samples

Populating the interactive namespace from numpy and matplotlib


In [2]:
Characteristics = pd.read_csv('Characteristics Sheet.csv')
Chemistry = pd.read_csv('Molecular Catalog.csv')

#merge the two sheets
output1 = pd.merge(Characteristics, Chemistry,
                  on='Star',
                  how='outer')

#read in specific columns
df = pd.DataFrame(output1, columns=['Star','Disk_has_Rings','HCN_flux','HCN_flux_uncertainty'])

#suppress rows with -99, this will leave us with only the targets where we had a detection for HCN
df_loc = df.loc[df['HCN_flux_uncertainty'] != -99]

print(df_loc)

          Star Disk_has_Rings      HCN_flux  HCN_flux_uncertainty
3       RU Lup           TRUE  3.690000e-17          3.900000e-18
9       AS 205          FALSE  2.200000e-16          1.950000e-17
12     DoAr 25           TRUE  4.880000e-17          1.700000e-18
17     WaOph 6           TRUE  1.710000e-17          2.000000e-18
18      AS 209           TRUE  6.800000e-17          1.510000e-17
20      FZ Tau            -99  5.690000e-17          8.720000e-18
21      TW Cha            -99  2.270000e-17          6.000000e-19
22      VZ Cha            -99  3.690000e-17          1.400000e-18
23      GQ Lup           TRUE  3.900000e-17          1.600000e-18
24  IRAS 04385            -99  2.240000e-17          3.420000e-18
26      CI Tau           TRUE  3.900000e-16          2.950000e-17
28      IQ Tau           TRUE  5.700000e-17          3.000000e-18


In [3]:
#read in specific columns
df = pd.DataFrame(output1, columns=['Star','Disk_has_Rings','C2H2_flux','C2H2_flux_uncertainty'])

#suppress rows with -99, this will leave us with only the targets where we had a detection for HCN
df_loc = df.loc[df['C2H2_flux_uncertainty'] != -99]

print(df_loc)

          Star Disk_has_Rings     C2H2_flux  C2H2_flux_uncertainty
9       AS 205          FALSE  2.110000e-16           1.500000e-17
12     DoAr 25           TRUE  9.900000e-18           1.300000e-18
18      AS 209           TRUE  2.790000e-17           6.140000e-18
20      FZ Tau            -99  3.530000e-17           8.530000e-18
21      TW Cha            -99  1.360000e-17           5.000000e-19
22      VZ Cha            -99  4.590000e-17           1.000000e-18
24  IRAS 04385            -99  9.450000e-18           2.100000e-18
26      CI Tau           TRUE  2.170000e-17           4.440000e-18
28      IQ Tau           TRUE  4.120000e-17           2.300000e-18


In [4]:
#read in specific columns
df = pd.DataFrame(output1, columns=['Star','Disk_has_Rings','CO2_flux','CO2_flux_uncertainty'])

#suppress rows with -99, this will leave us with only the targets where we had a detection for HCN
df_loc = df.loc[df['CO2_flux_uncertainty'] != -99]

print(df_loc)

          Star Disk_has_Rings      CO2_flux  CO2_flux_uncertainty
0       HT Lup          FALSE  5.580000e-17          1.080000e-17
1       GW Lup           TRUE  2.080000e-17          1.300000e-18
2       IM Lup           TRUE  8.600000e-18          1.500000e-18
3       RU Lup           TRUE  1.410000e-17          3.100000e-18
9       AS 205          FALSE  2.330000e-16          1.600000e-17
17     WaOph 6           TRUE  2.560000e-17          1.700000e-18
18      AS 209           TRUE  1.810000e-16          5.180000e-17
20      FZ Tau            -99  2.590000e-16          4.590000e-17
24  IRAS 04385            -99  3.260000e-17          7.610000e-18
26      CI Tau           TRUE  5.860000e-18          1.090000e-18


In [5]:
#seems to be a trend of the targets that don't have rings having some of the higher fluxes

#read in specific columns
df = pd.DataFrame(output1, columns=['Star','Disk_has_Rings','NeII_12.8_flux','NeII_12.8_flux_uncertainty'])

#suppress rows with -99, this will leave us with only the targets where we had a detection for HCN
df_loc = df.loc[df['NeII_12.8_flux_uncertainty'] != -99]

print(df_loc)

          Star Disk_has_Rings  NeII_12.8_flux  NeII_12.8_flux_uncertainty
2       IM Lup           TRUE    6.100000e-17                1.600000e-17
4       Sz 114           TRUE             NaN                         NaN
5       Sz 129           TRUE             NaN                         NaN
6       MY Lup           TRUE             NaN                         NaN
7    HD 142666           TRUE             NaN                         NaN
8    HD 143006           TRUE             NaN                         NaN
10        SR 4           TRUE             NaN                         NaN
11    Elias 20           TRUE             NaN                         NaN
12     DoAr 25           TRUE             NaN                         NaN
13    Elias 24           TRUE             NaN                         NaN
14    Elias 27           TRUE             NaN                         NaN
15     DoAr 33           TRUE             NaN                         NaN
16      WSB 52           TRUE         

In [6]:
#read in specific columns
df = pd.DataFrame(output1, columns=['Star','Disk_has_Rings','OH_23.2_microns','OH_23.2_microns_uncertainty'])

#suppress rows with -99, this will leave us with only the targets where we had a detection for HCN
df_loc = df.loc[df['OH_23.2_microns_uncertainty'] != -99]

print(df_loc)

         Star Disk_has_Rings  OH_23.2_microns  OH_23.2_microns_uncertainty
9      AS 205          FALSE     1.630000e-16                 5.800000e-18
17    WaOph 6           TRUE     4.610000e-17                 1.300000e-18
19  HD 163296           TRUE     4.860000e-15                 6.130000e-16
21     TW Cha            -99     1.640000e-17                 4.000000e-19
22     VZ Cha            -99     2.560000e-17                 6.000000e-19
23     GQ Lup           TRUE     3.260000e-17                 1.200000e-18


In [7]:
#read in specific columns
df = pd.DataFrame(output1, columns=['Star','Disk_has_Rings','OH_27.6_microns','OH_27.6_microns_uncertainty'])

#suppress rows with -99, this will leave us with only the targets where we had a detection for HCN
df_loc = df.loc[df['OH_27.6_microns_uncertainty'] != -99]

print(df_loc)

       Star Disk_has_Rings  OH_27.6_microns  OH_27.6_microns_uncertainty
9    AS 205          FALSE     4.770000e-16                 6.800000e-18
17  WaOph 6           TRUE     4.230000e-17                 1.500000e-18
21   TW Cha            -99     1.990000e-17                 5.000000e-19
22   VZ Cha            -99     2.790000e-17                 7.000000e-19
23   GQ Lup           TRUE     5.520000e-17                 1.400000e-18


In [9]:
#read in specific columns
df = pd.DataFrame(output1, columns=['Star','Disk_has_Rings','H2O_29.85_microns','H2O_29.85_uncertainty'])

#suppress rows with -99, this will leave us with only the targets where we had a detection for HCN
df_loc = df.loc[df['H2O_29.85_uncertainty'] != -99]

print(df_loc)

          Star Disk_has_Rings  H2O_29.85_microns  H2O_29.85_uncertainty
0       HT Lup          FALSE       1.170000e-14           1.500000e-18
3       RU Lup           TRUE       2.410000e-17           8.000000e-19
9       AS 205          FALSE       9.350000e-17           4.800000e-18
17     WaOph 6           TRUE       1.060000e-17           5.000000e-19
18      AS 209           TRUE       1.940000e-17           3.730000e-18
20      FZ Tau            -99       3.470000e-17           6.820000e-18
21      TW Cha            -99       5.500000e-18           2.000000e-19
22      VZ Cha            -99       7.500000e-18           2.000000e-19
23      GQ Lup           TRUE       1.330000e-17           4.000000e-19
24  IRAS 04385            -99       2.890000e-17           8.850000e-18
25      GK Tau          FALSE       2.740000e-17           6.660000e-18


In [10]:
#read in specific columns
df = pd.DataFrame(output1, columns=['Star','Disk_has_Rings','H2O_17.22_microns','H2O_17.22_uncertainty'])

#suppress rows with -99, this will leave us with only the targets where we had a detection for HCN
df_loc = df.loc[df['H2O_17.22_uncertainty'] != -99]

print(df_loc)

       Star Disk_has_Rings  H2O_17.22_microns  H2O_17.22_uncertainty
3    RU Lup           TRUE       3.490000e-17           1.400000e-18
9    AS 205          FALSE       1.820000e-16           7.400000e-18
17  WaOph 6           TRUE       1.540000e-17           7.000000e-19
20   FZ Tau            -99       6.580000e-17           2.220000e-18
21   TW Cha            -99       1.040000e-17           3.000000e-19
22   VZ Cha            -99       1.390000e-17           5.000000e-19
23   GQ Lup           TRUE       1.390000e-17           6.000000e-19
26   CI Tau           TRUE       1.010000e-17           2.870000e-18
28   IQ Tau           TRUE       9.000000e-18           9.000000e-19


In [11]:
#read in specific columns
df = pd.DataFrame(output1, columns=['Star','Disk_has_Rings','H2O_15.17_microns','H2O_15.17_uncertainty'])

#suppress rows with -99, this will leave us with only the targets where we had a detection for HCN
df_loc = df.loc[df['H2O_15.17_uncertainty'] != -99]

print(df_loc)

          Star Disk_has_Rings  H2O_15.17_microns  H2O_15.17_uncertainty
3       RU Lup           TRUE       2.500000e-17           1.300000e-18
9       AS 205          FALSE       1.160000e-16           6.500000e-18
17     WaOph 6           TRUE       1.570000e-17           7.000000e-19
20      FZ Tau            -99       4.500000e-17           3.850000e-18
21      TW Cha            -99       6.100000e-18           2.000000e-19
22      VZ Cha            -99       9.400000e-18           5.000000e-19
23      GQ Lup           TRUE       7.100000e-18           6.000000e-19
24  IRAS 04385            -99       6.050000e-18           1.650000e-18
25      GK Tau          FALSE       6.790000e-18           1.970000e-18
26      CI Tau           TRUE       1.310000e-17           1.410000e-18
28      IQ Tau           TRUE       5.700000e-18           1.000000e-18
